In [19]:
import pandas as pd
import numpy as np
import ssl
import urllib
from sklearn.compose import make_column_selector as selector

# Cargar los tres datasets:

In [20]:

# Crear un contexto SSL no verificado
context = ssl._create_unverified_context()

# Base URL para los datasets

base_url = "https://raw.githubusercontent.com/ricardoahumada/DataExpert/main/etapa4/data/"


# Función para descargar y cargar el dataset
def load_dataset(url):
    try:
        with urllib.request.urlopen(url, context=context) as response:
            return pd.read_csv(response)
    except Exception as e:
        print(f"Error al cargar los datos: {str(e)}")
        return None



In [21]:
# Cargar los datasets
df_original_caracteristicas_equipos = load_dataset(base_url + 'Caracteristicas_Equipos.csv')
df_original_historico_ordenes = load_dataset(base_url + 'Historicos_Ordenes.csv')
df_original_registros_condiciones = load_dataset(base_url + 'Registros_Condiciones.csv')

if df_original_caracteristicas_equipos is not None and df_original_historico_ordenes is not None and df_original_registros_condiciones is not None:
    print("Cargados correctamente.")
else:
    print("Error al cargar uno o más datasets.")


Cargados correctamente.


# Guardar copia en local:

In [22]:
# Los guardamos en local los originales para no depender.

df_original_caracteristicas_equipos.to_csv("../data/df_caracteristicas_equipos.csv", index=False)
df_original_historico_ordenes.to_csv("../data/df_historico_ordenes.csv", index=False)
df_original_registros_condiciones.to_csv("../data/df_registros_condiciones.csv", index=False)


# Cargar los tres datasets desde local:

In [23]:
# Cargar los tres datasets desde local:


try:
    df_trabajo_caracteristicas_equipos = pd.read_csv("../data/df_caracteristicas_equipos.csv")
    df_trabajo_historico_ordenes = pd.read_csv("../data/df_historico_ordenes.csv")
    df_trabajo_registros_condiciones = pd.read_csv("../data/df_registros_condiciones.csv")

except Exception as e:
    print(f"Error al cargar los datos: {str(e)}")


# Preprocesado por separado:

# Eliminar Nulos:

In [24]:
def eliminar_nulos(df):

# Esta función verifica si hay valores nulos en el DataFrame.
    
    nulos = df.isnull().sum().sum()
    print("Numero nulos=" + str(nulos))

    if nulos != 0:
        df.dropna(inplace=True)
        print("Valores nulos eliminados")
    else:
        print("No hay valores nulos")
    
    return

In [25]:
print(df_trabajo_caracteristicas_equipos.info())
print(df_trabajo_historico_ordenes.info())
print(df_trabajo_registros_condiciones.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   ID_Equipo                    500 non-null    int64 
 1   Tipo_Equipo                  500 non-null    object
 2   Fabricante                   500 non-null    object
 3   Modelo                       500 non-null    object
 4   Potencia_kW                  500 non-null    int64 
 5   Horas_Recomendadas_Revision  500 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 23.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61726 entries, 0 to 61725
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID_Orden             61726 non-null  int64 
 1   ID_Equipo            61726 non-null  int64 
 2   Fecha                61726 non-null  object
 3   Tipo_Mantenimiento   61726

In [26]:

eliminar_nulos(df_trabajo_caracteristicas_equipos)
eliminar_nulos(df_trabajo_historico_ordenes)
eliminar_nulos(df_trabajo_registros_condiciones)


Numero nulos=0
No hay valores nulos
Numero nulos=0
No hay valores nulos
Numero nulos=0
No hay valores nulos


# Eliminar duplicados:

In [27]:
def procesar_duplicados(df):

    # Identificar duplicados
    duplicados = df[df.duplicated()]
    print("Duplicados encontrados antes de la eliminación:")
    print(duplicados)

    # Contar duplicados por columna
    duplicados_count = df.duplicated().sum()
    print(f"Total de duplicados: {duplicados_count}")

    # Eliminar duplicados
    df.drop_duplicates(inplace=True)
    print("Duplicados eliminados.")

    return


In [28]:
procesar_duplicados(df_trabajo_caracteristicas_equipos)
procesar_duplicados(df_trabajo_historico_ordenes)
procesar_duplicados(df_trabajo_registros_condiciones)

Duplicados encontrados antes de la eliminación:
Empty DataFrame
Columns: [ID_Equipo, Tipo_Equipo, Fabricante, Modelo, Potencia_kW, Horas_Recomendadas_Revision]
Index: []
Total de duplicados: 0
Duplicados eliminados.
Duplicados encontrados antes de la eliminación:
Empty DataFrame
Columns: [ID_Orden, ID_Equipo, Fecha, Tipo_Mantenimiento, Costo_Mantenimiento, Duracion_Horas, Ubicacion]
Index: []
Total de duplicados: 0
Duplicados eliminados.
Duplicados encontrados antes de la eliminación:
Empty DataFrame
Columns: [ID_Registro, ID_Equipo, Fecha, Temperatura_C, Vibracion_mm_s, Horas_Operativas]
Index: []
Total de duplicados: 0
Duplicados eliminados.


# Inconsistencias numericas

In [29]:
def corregir_inconsistencias_numericas(df):
    # Selección de columnas numéricas
    numerical_columns_selector = selector(dtype_exclude=object)
    numerical_columns = numerical_columns_selector(df)

    # Corregir inconsistencias en campos numéricos
    for col_name in numerical_columns:
        df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
        df[col_name].fillna(df[col_name].median(), inplace=True)
        print(f"Inconsistencias corregidas en la columna numérica '{col_name}'.")

    return 

In [30]:
#corregir_inconsistencias_numericas(df_trabajo_caracteristicas_equipos)
#corregir_inconsistencias_numericas(df_trabajo_historico_ordenes)
#corregir_inconsistencias_numericas(df_trabajo_registros_condiciones)

# Outliers a efectos practicos:

In [31]:
def procesar_outliers(df):
  
    # Esta función identifica y winsoriza outliers en las columnas numéricas de un DataFrame.

    # Selección de columnas numéricas
    numerical_columns_selector = selector(dtype_exclude=object)
    numerical_columns = numerical_columns_selector(df)

    # Identificación de outliers
    IQR = df[numerical_columns].quantile(0.75) - df[numerical_columns].quantile(0.25)
    lower_bound = df[numerical_columns].quantile(0.25) - (IQR * 1.5)
    upper_bound = df[numerical_columns].quantile(0.75) + (IQR * 1.5)

    # Función para winsorizar una columna
    def winsorize_column(column, lower_bound, upper_bound):
        return column.clip(lower=lower_bound, upper=upper_bound)
    

    # Identificar y mostrar outliers antes de winsorización
    outliers = df[numerical_columns][(df[numerical_columns] < lower_bound) | (df[numerical_columns] > upper_bound)]
    print("Outliers encontrados antes de winsorización:")
    print(outliers)

# Mostrar la suma de outliers por columna
    outliers_count = outliers.count()
    print("Los outlayers encontrados, suma total por columna:")
    for col_name, count in outliers_count.items():
        print(f"{col_name}: {count}")

    # Procesar todas las columnas con outliers
    for col_name in numerical_columns:
        df[col_name] = winsorize_column(df[col_name], lower_bound[col_name], upper_bound[col_name])

    # Verificar outliers después de winsorización
    outliers = df[numerical_columns][(df[numerical_columns] < lower_bound) | (df[numerical_columns] > upper_bound)]
    print("Outliers - winsorized:")
    print(outliers.count())

    return

In [32]:
procesar_outliers(df_trabajo_caracteristicas_equipos)
procesar_outliers(df_trabajo_historico_ordenes)
procesar_outliers(df_trabajo_registros_condiciones)

Outliers encontrados antes de winsorización:
     ID_Equipo  Potencia_kW  Horas_Recomendadas_Revision
0          NaN          NaN                          NaN
1          NaN          NaN                          NaN
2          NaN          NaN                          NaN
3          NaN          NaN                          NaN
4          NaN          NaN                          NaN
..         ...          ...                          ...
495        NaN          NaN                          NaN
496        NaN          NaN                          NaN
497        NaN          NaN                          NaN
498        NaN          NaN                          NaN
499        NaN          NaN                          NaN

[500 rows x 3 columns]
Los outlayers encontrados, suma total por columna:
ID_Equipo: 0
Potencia_kW: 0
Horas_Recomendadas_Revision: 0
Outliers - winsorized:
ID_Equipo                      0
Potencia_kW                    0
Horas_Recomendadas_Revision    0
dtype: int64
Ou

# Transformacion fecha

In [33]:
def transformar_fechas(df):

    # Selección de columnas de tipo fecha
    date_columns_selector = selector(dtype_include='object')
    date_columns = date_columns_selector(df)

    # Transformar columnas de fecha a formato datetime
    for col_name in date_columns:
        try:
            df[col_name] = pd.to_datetime(df[col_name])
            print(f"Columna '{col_name}' transformada a formato datetime.")
        except Exception as e:
            print(f"Error al transformar la columna '{col_name}': {e}")

    return

Nuevas columnas, se me ocurren diferencias entre fechas de correctivos

# Informacion tabla ordenes: df_trabajo_historico_ordenes

# Nueva columna frecuencia de correctivo por equipo

In [34]:
# Filtrar solo los registros de mantenimiento correctivo
df_mantenimiento_correctivo = df_trabajo_historico_ordenes[df_trabajo_historico_ordenes['Tipo_Mantenimiento'] == 'Correctivo']

# Calcular la frecuencia de mantenimiento correctivo para cada equipo
df_frecuencia_mantenimiento_correctivo = df_mantenimiento_correctivo['ID_Equipo'].value_counts()

# Mapear la frecuencia al dataframe del histórico de órdenes
df_trabajo_historico_ordenes['Frecuencia de mantenimiento correctivo'] = df_trabajo_historico_ordenes['ID_Equipo'].map(df_frecuencia_mantenimiento_correctivo)


# Informacion tabla ordenes: df_trabajo_registros_condiciones

In [35]:
df_trabajo_registros_condiciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730500 entries, 0 to 730499
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID_Registro       730500 non-null  int64  
 1   ID_Equipo         730500 non-null  int64  
 2   Fecha             730500 non-null  object 
 3   Temperatura_C     730500 non-null  float64
 4   Vibracion_mm_s    730500 non-null  float64
 5   Horas_Operativas  730500 non-null  int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 33.4+ MB


# Grabado de fichero de trabajo

In [36]:
df_trabajo_caracteristicas_equipos.to_csv("../output/df_caracteristicas_equipos_tratados.csv", index=False)
df_trabajo_historico_ordenes.to_csv("../output/df_historico_ordenes_tratados.csv", index=False)
df_trabajo_registros_condiciones.to_csv("../output/df_registros_condiciones_tratados.csv", index=False)

# lectura ficheros de trabajo

In [37]:
try:
    df_trabajo_caracteristicas_equipos = pd.read_csv("../output/df_caracteristicas_equipos_tratados.csv")
    df_trabajo_historico_ordenes = pd.read_csv("../output/df_historico_ordenes_tratados.csv")
    df_trabajo_registros_condiciones = pd.read_csv("../output/df_registros_condiciones_tratados.csv")

except Exception as e:
    print(f"Error al cargar los datos: {str(e)}")


# Merge 

In [38]:

# Renombrar las columnas de fecha
df_trabajo_registros_condiciones.rename(columns={'Fecha': 'Fecha_Condiciones'}, inplace=True)
df_trabajo_historico_ordenes.rename(columns={'Fecha': 'Fecha_Ordenes'}, inplace=True)

# Convertir las columnas de fecha a formato datetime
df_trabajo_registros_condiciones['Fecha_Condiciones'] = pd.to_datetime(df_trabajo_registros_condiciones['Fecha_Condiciones'])
df_trabajo_historico_ordenes['Fecha_Ordenes'] = pd.to_datetime(df_trabajo_historico_ordenes['Fecha_Ordenes'])


# Fusionar los dataframes en 'ID_Equipo' y 'Fecha'
merged_df = pd.merge(df_trabajo_registros_condiciones, df_trabajo_caracteristicas_equipos, on='ID_Equipo', how='left')
merged_df = pd.merge(merged_df, df_trabajo_historico_ordenes, left_on=['ID_Equipo', 'Fecha_Condiciones'], right_on=['ID_Equipo', 'Fecha_Ordenes'], how='left')


In [39]:

# Eliminar las columnas especificadas
columnas_a_eliminar = ['Fecha_Ordenes', 'ID_Orden', 'Tipo_Equipo', 'ID_Registro','Potencia_kW', 'Fabricante', 'Modelo', 'Costo_Mantenimiento', 'Duracion_Horas', 'Ubicacion', 'Frecuencia de mantenimiento correctivo']
merged_df = merged_df.drop(columns=columnas_a_eliminar)


In [40]:

# Rellenar los valores NaN en la columna Tipo_Mantenimiento con "sin"
merged_df['Tipo_Mantenimiento'] = merged_df['Tipo_Mantenimiento'].fillna('sin')


# Guardado Final

In [41]:
# Guardar el dataframe final mezclado en un nuevo archivo CSV
merged_df.to_csv('../output/Datos_Fusionadosv1.csv', index=False)